# Custom Agents in Dana

This tutorial explores how to create custom agents in Dana by extending and specializing the base agent functionality. We'll learn how to create agents tailored for specific use cases and domains, focusing on the 2-layer architecture (planning and reasoning).

## Prerequisites

- Understanding of Dana basics (from [Introduction to Dana](../01_getting_started/01_introduction_to_dxa.ipynb))
- Knowledge of agent configuration (from [Agent Configuration](../01_getting_started/03_agent_configuration.ipynb))
- Familiarity with the core layers (from [Core Concepts](../02_core_concepts/))
- Dana package installed
- Python 3.8 or higher

In [ ]:
# Install Dana if you haven't already
%pip install dana

## 1. Understanding Agent Customization

In Dana's 2-layer architecture, agents can be customized in several ways:

1. **Specialization**: Creating agents for specific domains or tasks
2. **Extension**: Adding new capabilities to existing agents
3. **Composition**: Combining multiple agents into more complex systems
4. **Configuration**: Customizing agent behavior through settings

Let's explore each of these approaches.

In [ ]:
from dana import Agent, ChainOfThoughtStrategy, Plan


class CustomAgent(Agent):
    """Example of a custom agent with specialized planning and reasoning."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._domain_specific_knowledge = kwargs.get("domain_knowledge", {})
        self._custom_planning_strategy = kwargs.get("planning_strategy", None)
        self._custom_reasoning_strategy = kwargs.get("reasoning_strategy", None)

    async def plan(self, task: str) -> Plan:
        """Custom planning implementation."""
        # Use custom planning strategy if provided
        if self._custom_planning_strategy:
            return await self._custom_planning_strategy.plan(task)

        # Default planning behavior
        plan = Plan(name="custom_plan", description=f"Plan for task: {task}", objective=task)

        # Add domain-specific steps
        for step in self._get_domain_steps(task):
            plan.add_step(step)

        return plan

    async def reason(self, plan: Plan) -> dict:
        """Custom reasoning implementation."""
        # Use custom reasoning strategy if provided
        if self._custom_reasoning_strategy:
            return await self._custom_reasoning_strategy.reason(plan)

        # Default reasoning behavior
        strategy = ChainOfThoughtStrategy()
        return await strategy.reason(plan)

    def _get_domain_steps(self, task: str) -> list:
        """Get domain-specific plan steps."""
        # Example implementation
        return [
            {"name": "domain_analysis", "description": "Analyze task in domain context", "tool": "analyze", "tool_params": {"task": task}}
        ]


# Create a custom agent
agent = CustomAgent(
    name="specialized_agent",
    description="Agent specialized for a specific domain",
    domain_knowledge={"domain": "manufacturing", "capabilities": ["quality_control", "process_optimization"]},
)

# Test the custom agent
response = agent.ask("Analyze this manufacturing process")
print(response["result"])

## 2. Extending Agent Capabilities

Let's see how to extend agent capabilities:

In [ ]:
from dana import BaseResource, ResourceSelector


class ExtendedAgent(Agent):
    """Example of an agent with extended capabilities."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._extended_resources = kwargs.get("extended_resources", [])
        self._resource_selector = ResourceSelector(self.execution_context)

    async def execute_task(self, task: dict) -> dict:
        """Execute task with extended capabilities."""
        # Select appropriate resource
        resource = await self._resource_selector.select_resource(task)

        # Execute task with selected resource
        result = await self._execute_with_resource(resource, task)

        # Apply extended processing
        return await self._apply_extended_processing(result)

    async def _execute_with_resource(self, resource: BaseResource, task: dict) -> dict:
        """Execute task with selected resource."""
        try:
            method = getattr(resource, task["operation"])
            return await method(**task["parameters"])
        except Exception as e:
            return {"success": False, "error": str(e)}

    async def _apply_extended_processing(self, result: dict) -> dict:
        """Apply extended processing to result."""
        # Example implementation
        if result.get("success"):
            result["extended_analysis"] = {"timestamp": "2024-04-19", "confidence": 0.95}
        return result


# Create an extended agent
agent = ExtendedAgent(
    name="extended_agent",
    description="Agent with extended capabilities",
    extended_resources=[
        {"name": "analysis", "capabilities": ["analyze", "optimize"]},
        {"name": "visualization", "capabilities": ["visualize", "report"]},
    ],
)

# Test the extended agent
response = agent.ask("Analyze and visualize this data")
print(response["result"])

## 3. Composing Multiple Agents

Let's see how to compose multiple agents:

In [ ]:
class ComposedAgent(Agent):
    """Example of an agent composed of multiple specialized agents."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._specialized_agents = kwargs.get("specialized_agents", {})

    async def execute_complex_task(self, task: dict) -> dict:
        """Execute complex task using multiple specialized agents."""
        # Decompose task into subtasks
        subtasks = self._decompose_task(task)

        # Execute subtasks with appropriate agents
        results = {}
        for subtask in subtasks:
            agent = self._select_agent(subtask)
            results[subtask["name"]] = await agent.execute_task(subtask)

        # Combine results
        return self._combine_results(results)

    def _decompose_task(self, task: dict) -> list:
        """Decompose task into subtasks."""
        # Example implementation
        return [
            {"name": "analysis", "operation": "analyze", "parameters": task},
            {"name": "optimization", "operation": "optimize", "parameters": task},
        ]

    def _select_agent(self, subtask: dict) -> Agent:
        """Select appropriate agent for subtask."""
        # Example implementation
        agent_type = subtask["name"]
        return self._specialized_agents.get(agent_type, self)

    def _combine_results(self, results: dict) -> dict:
        """Combine results from multiple agents."""
        # Example implementation
        return {"success": all(r.get("success") for r in results.values()), "results": results}


# Create specialized agents
analysis_agent = CustomAgent(name="analysis_agent", description="Agent specialized for analysis", domain_knowledge={"domain": "analysis"})

optimization_agent = CustomAgent(
    name="optimization_agent", description="Agent specialized for optimization", domain_knowledge={"domain": "optimization"}
)

# Create composed agent
agent = ComposedAgent(
    name="composed_agent",
    description="Agent composed of specialized agents",
    specialized_agents={"analysis": analysis_agent, "optimization": optimization_agent},
)

# Test the composed agent
response = agent.ask("Analyze and optimize this process")
print(response["result"])

## 4. Testing Custom Agents

Let's see how to test custom agents:

In [ ]:
import pytest

# Test data
TEST_TASK = {"operation": "analyze", "parameters": {"data": [1, 2, 3, 4, 5]}}


# Test custom agent
async def test_custom_agent():
    agent = CustomAgent(name="test_agent", description="Test agent", domain_knowledge={"domain": "test"})

    # Test planning
    plan = await agent.plan("test task")
    assert plan.name == "custom_plan"
    assert len(plan.steps) > 0

    # Test reasoning
    result = await agent.reason(plan)
    assert "result" in result


# Test extended agent
async def test_extended_agent():
    agent = ExtendedAgent(name="test_agent", description="Test agent", extended_resources=[{"name": "test", "capabilities": ["test"]}])

    # Test task execution
    result = await agent.execute_task(TEST_TASK)
    assert "success" in result
    assert "extended_analysis" in result


# Test composed agent
async def test_composed_agent():
    agent = ComposedAgent(name="test_agent", description="Test agent", specialized_agents={"test": CustomAgent(name="test")})

    # Test complex task execution
    result = await agent.execute_complex_task(TEST_TASK)
    assert "success" in result
    assert "results" in result


# Run the tests
if __name__ == "__main__":
    pytest.main([__file__])

## Next Steps

In this tutorial, we've covered:

1. Understanding agent customization in the 2-layer architecture
2. Extending agent capabilities
3. Composing multiple agents
4. Testing custom agents

In the next tutorial, we'll explore advanced workflows in Dana.